# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import sys
from sqlalchemy import create_engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.metrics import classification_report
from sklearn import multioutput
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
# load data from database
engine = create_engine('sqlite:///disasters_responses.db')
df = pd.read_sql_table('disasters_responses', engine)

In [15]:
engine = create_engine('sqlite:///disasters_responses.db')
dataset = pd.read_sql_table('disasters_responses', engine)

In [16]:
X = df['message']
Y = df.iloc[:, 3:]
category_names = Y.columns

### 2. Write a tokenization function to process your text data

In [17]:
def tokenize(text):
    '''
    input:
    text - string that needs to be tokenized
    outout:
    a list of words
    '''
    #normalize the text
    text = text.lower()
    #Remove punctuation characters
    text = re.sub(r'(?:([A-Za-z0-9]+)/([A-Za-z0-9]+))|(/)', " ", text)
    text = re.sub(r'[?|$|.|!]', r'', text)
    # text = r'(?:([A-Za-z0-9]+)/([A-Za-z0-9]+))|(/)'
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [i for i in tokens if i not in stopwords.words("english")]
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
def build_model():
    pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

    ])),

    ('clf', RandomForestClassifier())
    ])    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [30]:
# Y_test

In [31]:
X_train.shape

(20972,)

In [32]:
X_test.shape

(5244,)

In [33]:
Y_train.shape

(20972, 36)

In [34]:
Y_test.shape

(5244, 36)

In [39]:
model = build_model()

In [56]:
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [57]:
Y_pred = model.predict(X_test)

In [80]:
Y_pred

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [63]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [65]:
Y_test.values

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 0]])

In [77]:
Y_test.values[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [70]:
print(type(Y_test.values), type(Y_pred))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [73]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [74]:
Y.columns.values 

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'], dtype=object)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [72]:
for i, col in enumerate(category_names):
        print(col, classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

related              precision    recall  f1-score   support

          0       0.57      0.47      0.51      1227
          1       0.84      0.89      0.87      3984
          2       0.38      0.09      0.15        33

avg / total       0.77      0.79      0.78      5244

request              precision    recall  f1-score   support

          0       0.90      0.98      0.94      4396
          1       0.82      0.41      0.54       848

avg / total       0.88      0.89      0.87      5244

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5219
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      5244

aid_related              precision    recall  f1-score   support

          0       0.70      0.91      0.79      3104
          1       0.77      0.43      0.55      2140

avg / total       0.73      0.71      0.69      5244

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
#We need to add parameters to ramdonforest classifier to make it more precise and see 
#if improves our model - try as many as you want yet running (fitting) this is time 
#consumming. Here we use GridSearchCV
pipeline_parametized = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
    ])),

     ('clf', multioutput.MultiOutputClassifier(RandomForestClassifier( random_state=0)))
])

parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 500),
#         'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [20],
        'clf__estimator__min_samples_split': [2, 3]
#         'clf__estimator__max_depth': [3]
#         'estimator__features__n_jobs': [2]
#         'estimator__clf__estimator__max_features': (None, 5000)
#         'estimator__clf__estimator__max_features': [None, 5000]
#         'estimator__clf__estimator__bootstrap': [True]
    }
cv = GridSearchCV(pipeline_parametized, param_grid = parameters)
    
    

In [159]:
#we get an error about wrong parameters, we fixed by looking at the right 
#parameters accepted
pipeline_parametized.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              

In [22]:
#traing the model
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [20], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
Y_pred_parametized = cv.predict(X_test)

In [29]:
for i, col in enumerate(category_names):
        print(col, classification_report(Y_test.iloc[:,i], Y_pred_parametized[:,i]))

related              precision    recall  f1-score   support

          0       0.67      0.29      0.40      1272
          1       0.80      0.95      0.87      3930
          2       0.44      0.19      0.27        42

avg / total       0.77      0.79      0.75      5244

request              precision    recall  f1-score   support

          0       0.89      0.99      0.94      4339
          1       0.88      0.41      0.56       905

avg / total       0.89      0.89      0.87      5244

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5228
          1       0.00      0.00      0.00        16

avg / total       0.99      1.00      1.00      5244

aid_related              precision    recall  f1-score   support

          0       0.75      0.87      0.81      3081
          1       0.76      0.58      0.66      2163

avg / total       0.75      0.75      0.74      5244

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


We did somewhat better that the past model by examining the avg/totals for each category and making a comparison. Adding more parameters might improve this model to a greater extent, but we stopped trying because it takes too loong to train the model.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [33]:
# model_pkl = pickle.dump(cv, 'classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.